# Implementation of Knowledge Based SDDC LCM. 

#### Proposing a Knowledge Based Life Cycle Management method for the Infrastructure Components and the Applications in the Scaled Hyper Converged Software Defined Data Centers is a project to manage life cycle Management (LCM) software which will look for “Best Window time” or “best time” to perform Life Cycle Management.  So that the Administrator can perform LCM whenever upgrade or patch is available for any entity (Application or Infrastructure component) .  This will help avoid any risk because of ongoing  upgrades or patches 

### Import Library

In [683]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
import re
import time
from time import strftime
from utils import util


In [684]:
__author__ = 'raviranjan'

### Define variables contants

In [685]:
LOG_DIR = "logs"
HOST_PATTERN = r'(^\S+\.[\S+\.]+\S+)\s'
LOGGER_FILE_PATTERN = r'(^\S+\.[\S+\.]+\S+)\s'
ERROR_PATTERN = 'ERROR:'
INFO_PATTERN = 'INFO:'
DEBUG_PATTERN = 'DEBUG:'
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
logger_file_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
has_error_pattern = r''
excel_file_path = "data/excel_data_1.xlsx"



# Create a util class

In [686]:
import re
import os
import time
from time import strftime


def export_to_file(dict_with_list_values=None, export_file_path=None, export_folder=None):
    """
    dict_with_list_values = {'Brand': ['Honda Civic','Toyota Corolla','Ford Focus','Audi A4'],
        'Price': [32000,35000,37000,45000]
        }
    :param export_folder:
    :param dict_with_list_values:
    :param create_dict_from_list:
    :param column_name
    :return:
    """
    if not export_file_path:
        time_now = str(strftime("%Y-%m-%d %H-%M-%S", time.localtime()))
        export_file_path = export_folder + "/" + time_now + ".xlsx"
    columns = dict_with_list_values.keys()
    print('Exporting columns %s to %s' % (columns, export_file_path))
    import pandas as pd
    df = pd.DataFrame(dict_with_list_values, columns=columns)
    df.to_excel(export_file_path, index=False, header=True)
    print(export_file_path)
    return export_file_path


def extract_data_from_logs(log_files=None,
                           column_names=['time', 'info', 'error', 'debug', 'warn', 'exception',
                                         'filename', 'line'], add_time_stamp=True):
    excel_export_data = {
    }
    for column_name in column_names:
        excel_export_data[column_name] = []
    print("Define column %s" % excel_export_data)
    for log_file in log_files:
        with open(log_file) as f:
            for line in f:
                one_row_values = {}  # store one row value with key as column name and its value
                for column_name in column_names:
                    column_name = column_name.lower()
                    line = line.lower().strip()
                    if column_name in line:
                        cell_value = 1
                    else:
                        cell_value = 0
                    if column_name == "line":
                        cell_value = line
                    if column_name == "filename":
                        cell_value = log_file.split('/')[-1]
                    if add_time_stamp and column_name == "time":
                        RE_COMPILE_TIME_STAMP_PATTERN = "(24:00|2[0-3]:[0-5][0-9]|[0-1][0-9]:[0-5][0-9]:[0-5][0-9])"
                        cell_value = finditer_line(line, regex=RE_COMPILE_TIME_STAMP_PATTERN,
                                                   log_file=log_file)
                        if not cell_value:
                            line = replace_timestamp_with_time(line)
                            cell_value = finditer_line(line, regex=RE_COMPILE_TIME_STAMP_PATTERN,
                                                       log_file=log_file)
                        if not cell_value:
                            #TODO reparse
                            pass

                    # Final cell value for that column
                    one_row_values[column_name] = cell_value

                for column_name in one_row_values:
                    # append the list value
                    existing_one_column_values = excel_export_data[column_name]
                    existing_one_column_values.append(one_row_values[column_name])
                    excel_export_data[column_name] = existing_one_column_values
    files = []
    for log_file in log_files:
        log_file = log_file.split('/')[-1]
        files.append(log_file)
    for column_name in column_names:
        print("Total %s values for Column:%s in all log files" % (
        len(excel_export_data[column_name]), column_name))

    return excel_export_data


def get_log_files(log_dir):
    log_files = []
    for root, dir_, files in os.walk(log_dir):
        for sub_dir in dir_:
            sub_dir_path = os.path.join(root, sub_dir)
            # print("sub_dir: %s" % sub_dir_path)
            # os.system('ls %s' % sub_dir_path)
        for f in files:
            f_path = os.path.join(root, f)
            if ".log" in f_path:
                log_files.append(f_path)
    print("%s" % (log_files))
    return log_files


def findall_(log_file, regex):
    matched_lines = []
    with open(log_file) as f:
        # lines = f.readlines()
        for line in f:
            found = re.findall(regex, line)
            if found:
                matched_lines.append(line)
        print(matched_lines)
    return matched_lines


def search_re(line, pattern=None):
    # Python program to illustrate
    # Matching regex objects
    match = re.match(pattern, line)
    if not match:
        regex_com = re.compile(pattern)
        match = regex_com.search(line)
        if match:
            value = match.group()
            print('found using re.compile: %s' % value)
            return value
        else:
            print('%s not Found using re.compile also:' % pattern)
            return ''
    else:
        value = match.group()
        print('Found using re.match: %s' % value)
        return value


def finditer_(log_file_path, regex, read_line=True):
    """
    regex = '(<property name="(.*?)">(.*?)<\/property>)'
    :param log_file_path:
    :param regex:
    :param read_line:
    :param re_parse:
    :return:
    """
    with open(log_file_path, "r") as f:
        match_list = []
        if read_line:
            for line in f:
                for match in re.finditer(regex, line, re.S):
                    match_text = match.group()
                    match_list.append(match_text)
        else:
            data = f.read()
            for match in re.finditer(regex, data, re.S):
                match_text = match.group()
                match_list.append(match_text)
    f.close()
    print(match_list)
    return match_list


def finditer_line(line, regex, log_file=None):
    """
    regex = '(<property name="(.*?)">(.*?)<\/property>)'
    :param log_file_path:
    :param regex:
    :param read_line:
    :param re_parse:
    :return:
    """
    match_text = ''
    for match in re.finditer(regex, line, re.S):
        match_text = match.group()
    return match_text


def count_cell_entries(df, col_name='', output_col_name=''):
    """
    Returns pd.value_counts() as a DataFrame

    Parameters
    ----------
    df : Pandas Dataframe
        Dataframe on which to run value_counts(), must have column `col`.
    col : str
        Name of column in `df` for which to generate counts

    Returns
    -------
    Pandas Dataframe
        Returned dataframe will have a single column named "count" which contains the count_values()
        for each unique value of df[col]. The index name of this dataframe is `col`.

    Example
    -------
    >>> value_counts_df(pd.DataFrame({'a':[1, 1, 2, 2, 2]}), 'a')
       count
    a
    2      3
    1      2
    """
    # df = pd.DataFrame(df[col].value_counts())
    # df.index.name = col
    # df.columns = ['count']
    z = df[col_name].value_counts()
    z1 = z.to_dict()  # converts to dictionary
    df[output_col_name] = df[col_name].map(z1)
    return df


def date2int(df):
    if df.timestamp:
        t = df['time']
        try:
            t1 = t.timetuple()
            return int(time.mktime(t1))
        except ValueError:
            return None


def timestamp2date(timestamp):
    from datetime import datetime
    return datetime.fromtimestamp(int(timestamp))


def replace_timestamp_with_time(line):
    if line.startswith('-'):
        tmp = line[line.index('-') + len('-'):].strip()
    elif line.startswith(' -'):
        tmp = line[line.index('-') + len('-'):].strip()
    else:
        tmp = line
    tmp = tmp.split(" ")
    if tmp:
        tmp = tmp[0]
        if tmp.isnumeric():
            time = timestamp2date(tmp)
            line = line.replace(tmp, str(time))
    return line


### Get all components log files

In [687]:
log_files = get_log_files(log_dir=LOG_DIR)

['logs/HDFS/HDFS.log', 'logs/Windows/Windows.log', 'logs/Apache/Apache.log', 'logs/HealthApp/HealthApp.log', 'logs/Thunderbird/Thunderbird.log', 'logs/Spark/Spark.log', 'logs/Mac/Mac.log', 'logs/BGL/BGL.log', 'logs/Andriod/Andriod.log', 'logs/Hadoop/Hadoop.log', 'logs/Proxifier/Proxifier.log', 'logs/HPC/HPC.log', 'logs/Linux/Linux.log', 'logs/Zookeeper/Zookeeper.log', 'logs/OpenStack/OpenStack.log', 'logs/OpenSSH/SSH.log']


### read logs and dumps/export to Excel 

In [688]:
excel_export_data = extract_data_from_logs(log_files=log_files, column_names=['time', 'info', 'error', 'debug', 'warn', 'exception', 'filename', 'line'])

Define column {'time': [], 'info': [], 'error': [], 'debug': [], 'warn': [], 'exception': [], 'filename': [], 'line': []}
Total 32000 values for Column:time in all log files
Total 32000 values for Column:info in all log files
Total 32000 values for Column:error in all log files
Total 32000 values for Column:debug in all log files
Total 32000 values for Column:warn in all log files
Total 32000 values for Column:exception in all log files
Total 32000 values for Column:filename in all log files
Total 32000 values for Column:line in all log files


In [689]:
#export_to_file(dict_with_list_values=excel_export_data, export_file_path=excel_file_path)

# Analyzing Data

## Read the Exported Excel file

In [690]:
import time
print(excel_file_path)
df = pd.read_excel(excel_file_path)

data/excel_data_1.xlsx


In [691]:
df

,time,info,error,debug,warn,exception,filename,line
0,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:606|step_lsc|30002312|onstan...
1,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:615|step_lsc|30002312|onexte...
2,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:633|step_standreportreceiver...
3,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_lsc|30002312|proces...
4,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_standstepcounter|30...
...,...,...,...,...,...,...,...,...
31995,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info storage.blockmanager: f...
31996,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: fini...
31997,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.coarsegrainede...
31998,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: runn...


## Add time along with  timestamp

In [692]:
# create a new column time and conver timestamp to time
df["timestamp"] = [time.mktime(t.timetuple()) if not pd.isnull(t) else None  for t in pd.to_datetime(df['time'], format='%M:%S', errors='coerce')]
df

,time,info,error,debug,warn,exception,filename,line,timestamp
0,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:606|step_lsc|30002312|onstan...,-2.208987e+09
1,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:615|step_lsc|30002312|onexte...,-2.208987e+09
2,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:633|step_standreportreceiver...,-2.208987e+09
3,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_lsc|30002312|proces...,-2.208987e+09
4,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_standstepcounter|30...,-2.208987e+09
...,...,...,...,...,...,...,...,...,...
31995,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info storage.blockmanager: f...,-2.208988e+09
31996,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: fini...,-2.208988e+09
31997,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.coarsegrainede...,-2.208988e+09
31998,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: runn...,-2.208988e+09


## Count , how many logs entries of given time

## Hold the value in time_count

In [693]:
count_cell_entries(df, col_name='time', output_col_name='time_count')


,time,info,error,debug,warn,exception,filename,line,timestamp,time_count
0,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:606|step_lsc|30002312|onstan...,-2.208987e+09,318.0
1,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:615|step_lsc|30002312|onexte...,-2.208987e+09,318.0
2,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:633|step_standreportreceiver...,-2.208987e+09,318.0
3,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_lsc|30002312|proces...,-2.208987e+09,318.0
4,22:15,0,0,0,0,0,HealthApp.log,20171223-22:15:29:635|step_standstepcounter|30...,-2.208987e+09,318.0
...,...,...,...,...,...,...,...,...,...,...
31995,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info storage.blockmanager: f...,-2.208988e+09,923.0
31996,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: fini...,-2.208988e+09,923.0
31997,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.coarsegrainede...,-2.208988e+09,923.0
31998,20:11,1,0,0,0,0,Spark.log,17/06/09 20:11:11 info executor.executor: runn...,-2.208988e+09,923.0


### sort by timestamp_count

In [694]:
df = df.sort_values(by=['time_count'], ascending=False)
df

,time,info,error,debug,warn,exception,filename,line,timestamp,time_count
30072,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:48 info storage.memorystore: bl...,-2.208988e+09,1108.0
30665,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:56 info python.pythonrunner: ti...,-2.208988e+09,1108.0
30652,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info executor.coarsegrainede...,-2.208988e+09,1108.0
30653,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info executor.executor: runn...,-2.208988e+09,1108.0
30654,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info broadcast.torrentbroadc...,-2.208988e+09,1108.0
...,...,...,...,...,...,...,...,...,...,...
23972,NaN,0,1,0,0,0,BGL.log,appout 1134357549 2005.12.11 r63-m1-n0-i:j18-u...,NaN,NaN
23973,NaN,0,1,0,0,0,BGL.log,appsev 1134496355 2005.12.13 r74-m1-n8-i:j18-u...,NaN,NaN
23974,NaN,0,1,0,0,0,BGL.log,appsev 1134496398 2005.12.13 r44-m0-n4-i:j18-u...,NaN,NaN
23980,NaN,0,1,0,0,0,BGL.log,appsev 1134630981 2005.12.14 r37-m1-n8-i:j18-u...,NaN,NaN


#  Data Wrangling


## Data Wrangling is to remove null or empty data, cleaning data set

In [695]:
print((df.info()))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 30072 to 23981
Data columns (total 10 columns):
time          31613 non-null object
info          32000 non-null int64
error         32000 non-null int64
debug         32000 non-null int64
warn          32000 non-null int64
exception     32000 non-null int64
filename      32000 non-null object
line          32000 non-null object
timestamp     6141 non-null float64
time_count    31613 non-null float64
dtypes: float64(2), int64(5), object(3)
memory usage: 2.7+ MB
None


In [696]:
df.isnull().sum()



time            387
info              0
error             0
debug             0
warn              0
exception         0
filename          0
line              0
timestamp     25859
time_count      387
dtype: int64

## either drop null row or fill some value where value is null


In [697]:
#df.drop("line", axis=1, inplace=True)

### Replace NaN Values with Zeros

In [698]:
df = df.fillna(0)
df


,time,info,error,debug,warn,exception,filename,line,timestamp,time_count
30072,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:48 info storage.memorystore: bl...,-2.208988e+09,1108.0
30665,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:56 info python.pythonrunner: ti...,-2.208988e+09,1108.0
30652,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info executor.coarsegrainede...,-2.208988e+09,1108.0
30653,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info executor.executor: runn...,-2.208988e+09,1108.0
30654,20:10,1,0,0,0,0,Spark.log,17/06/09 20:10:55 info broadcast.torrentbroadc...,-2.208988e+09,1108.0
...,...,...,...,...,...,...,...,...,...,...
23972,0,0,1,0,0,0,BGL.log,appout 1134357549 2005.12.11 r63-m1-n0-i:j18-u...,0.000000e+00,0.0
23973,0,0,1,0,0,0,BGL.log,appsev 1134496355 2005.12.13 r74-m1-n8-i:j18-u...,0.000000e+00,0.0
23974,0,0,1,0,0,0,BGL.log,appsev 1134496398 2005.12.13 r44-m0-n4-i:j18-u...,0.000000e+00,0.0
23980,0,0,1,0,0,0,BGL.log,appsev 1134630981 2005.12.14 r37-m1-n8-i:j18-u...,0.000000e+00,0.0


In [699]:
df.isnull().sum()

time          0
info          0
error         0
debug         0
warn          0
exception     0
filename      0
line          0
timestamp     0
time_count    0
dtype: int64

## Drop the info debug and warn having value 1

In [700]:

#df.drop(df.loc[df['info'] == 1].index, inplace=True)
#df.drop(df.loc[df['debug'] == 1].index, inplace=True)
#df.drop(df.loc[df['warn'] == 1].index, inplace=True)
# df.drop(df.loc[df['error'] == 0].index, inplace=True)

In [701]:
df = df.loc[(df['error'] == 1) | (df['exception'] == 1)]
df

,time,info,error,debug,warn,exception,filename,line,timestamp,time_count
9448,20:10,0,1,0,0,0,HPC.log,1970-01-30 20:10:15 interconnect-1t02 switch_m...,-2.208988e+09,1108.0
9619,20:10,0,1,0,0,0,HPC.log,1970-01-02 20:10:25 interconnect-1n01 switch_m...,-2.208988e+09,1108.0
9620,20:10,0,1,0,0,0,HPC.log,1970-01-02 20:10:38 interconnect-1n02 switch_m...,-2.208988e+09,1108.0
20790,04:01:50,0,0,0,1,1,HDFS.log,1970-01-02 04:01:50 175354 14021 warn dfs.data...,0.000000e+00,952.0
20794,04:01:50,0,0,0,1,1,HDFS.log,1970-01-02 04:01:50 175934 12298 warn dfs.data...,0.000000e+00,952.0
...,...,...,...,...,...,...,...,...,...,...
23972,0,0,1,0,0,0,BGL.log,appout 1134357549 2005.12.11 r63-m1-n0-i:j18-u...,0.000000e+00,0.0
23973,0,0,1,0,0,0,BGL.log,appsev 1134496355 2005.12.13 r74-m1-n8-i:j18-u...,0.000000e+00,0.0
23974,0,0,1,0,0,0,BGL.log,appsev 1134496398 2005.12.13 r44-m0-n4-i:j18-u...,0.000000e+00,0.0
23980,0,0,1,0,0,0,BGL.log,appsev 1134630981 2005.12.14 r37-m1-n8-i:j18-u...,0.000000e+00,0.0


In [702]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2575 entries, 9448 to 23981
Data columns (total 10 columns):
time          2575 non-null object
info          2575 non-null int64
error         2575 non-null int64
debug         2575 non-null int64
warn          2575 non-null int64
exception     2575 non-null int64
filename      2575 non-null object
line          2575 non-null object
timestamp     2575 non-null float64
time_count    2575 non-null float64
dtypes: float64(2), int64(5), object(3)
memory usage: 221.3+ KB


## Get the time_count for every one hour interval

### create two new column

In [703]:
df['time_range'] = 0
df['time_range_count'] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [704]:
df.drop("line", axis=1, inplace=True)
df


,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
9448,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,0,0
9619,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,0,0
9620,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,0,0
20790,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,0,0
20794,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
23972,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23973,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23974,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23980,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0


### Repalce time with one hour interval range

In [705]:

df.loc[df['time'].str.startswith("00:") == True, 'time_range'] = '00:00:00-01:00:00'
df.loc[df['time'].str.startswith("01:") == True, 'time_range'] = '01:00:00-02:00:00'
df.loc[df['time'].str.startswith("02:") == True, 'time_range'] = '02:00:00-03:00:00'
df.loc[df['time'].str.startswith("03:") == True, 'time_range'] = '03:00:00-04:00:00'
df.loc[df['time'].str.startswith("04:") == True, 'time_range'] = '04:00:00-05:00:00'
df.loc[df['time'].str.startswith("05:") == True, 'time_range'] = '05:00:00-06:00:00'
df.loc[df['time'].str.startswith("06:") == True, 'time_range'] = '06:00:00-07:00:00'
df.loc[df['time'].str.startswith("07:") == True, 'time_range'] = '07:00:00-08:00:00'
df.loc[df['time'].str.startswith("08:") == True, 'time_range'] = '08:00:00-09:00:00'
df.loc[df['time'].str.startswith("09:") == True, 'time_range'] = '09:00:00-10:00:00'
df.loc[df['time'].str.startswith("10:") == True, 'time_range'] = '10:00:00-11:00:00'
df.loc[df['time'].str.startswith("11:") == True, 'time_range'] = '11:00:00-12:00:00'
df.loc[df['time'].str.startswith("12:") == True, 'time_range'] = '12:00:00-13:00:00'
df.loc[df['time'].str.startswith("13:") == True, 'time_range'] = '13:00:00-14:00:00'
df.loc[df['time'].str.startswith("14:") == True, 'time_range'] = '14:00:00-15:00:00'
df.loc[df['time'].str.startswith("15:") == True, 'time_range'] = '15:00:00-16:00:00'
df.loc[df['time'].str.startswith("16:") == True, 'time_range'] = '16:00:00-17:00:00'
df.loc[df['time'].str.startswith("17:") == True, 'time_range'] = '17:00:00-18:00:00'
df.loc[df['time'].str.startswith("18:") == True, 'time_range'] = '18:00:00-19:00:00'
df.loc[df['time'].str.startswith("19:") == True, 'time_range'] = '19:00:00-20:00:00'
df.loc[df['time'].str.startswith("20:") == True, 'time_range'] = '20:00:00-21:00:00'
df.loc[df['time'].str.startswith("21:") == True, 'time_range'] = '21:00:00-22:00:00'
df.loc[df['time'].str.startswith("22:") == True, 'time_range'] = '22:00:00-23:00:00'
df.loc[df['time'].str.startswith("23:") == True, 'time_range'] = '23:00:00-24:00:00'
df.loc[df['time'].str.startswith("24:") == True, 'time_range'] = '24:00:00-00:00:00'


In [706]:
df

,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
9448,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,0
9619,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,0
9620,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,0
20790,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,04:00:00-05:00:00,0
20794,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,04:00:00-05:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...
23972,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23973,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23974,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0
23980,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,0


## count the time_range value counts

In [707]:
count_cell_entries(df, col_name='time_range', output_col_name='time_range_count')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
9448,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,114
9619,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,114
9620,20:10,0,1,0,0,0,HPC.log,-2.208988e+09,1108.0,20:00:00-21:00:00,114
20790,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,04:00:00-05:00:00,195
20794,04:01:50,0,0,0,1,1,HDFS.log,0.000000e+00,952.0,04:00:00-05:00:00,195
...,...,...,...,...,...,...,...,...,...,...,...
23972,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,90
23973,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,90
23974,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,90
23980,0,0,1,0,0,0,BGL.log,0.000000e+00,0.0,0,90


In [708]:
df = df.sort_values(by=['time_range_count'], ascending=True)
df

,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
23975,08:52:16,1,1,0,0,0,BGL.log,0.0,1.0,08:00:00-09:00:00,13
23993,08:47:22,1,1,0,0,0,BGL.log,0.0,2.0,08:00:00-09:00:00,13
26580,08:54:17,0,1,0,0,0,Apache.log,0.0,1.0,08:00:00-09:00:00,13
13398,08:35:08,0,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
13392,08:32:40,1,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
...,...,...,...,...,...,...,...,...,...,...,...
11088,19:32:24,0,1,0,1,0,Zookeeper.log,0.0,4.0,19:00:00-20:00:00,374
7628,19:01:46,0,1,0,0,0,Proxifier.log,0.0,2.0,19:00:00-20:00:00,374
7629,19:01:46,0,1,0,0,0,Proxifier.log,0.0,2.0,19:00:00-20:00:00,374
11006,19:29:40,0,1,0,1,0,Zookeeper.log,0.0,4.0,19:00:00-20:00:00,374


## Get minimum log count

In [709]:
minvalue = df['time_range_count'].min()

In [710]:
print(minvalue)

13


In [711]:
df

,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
23975,08:52:16,1,1,0,0,0,BGL.log,0.0,1.0,08:00:00-09:00:00,13
23993,08:47:22,1,1,0,0,0,BGL.log,0.0,2.0,08:00:00-09:00:00,13
26580,08:54:17,0,1,0,0,0,Apache.log,0.0,1.0,08:00:00-09:00:00,13
13398,08:35:08,0,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
13392,08:32:40,1,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
...,...,...,...,...,...,...,...,...,...,...,...
11088,19:32:24,0,1,0,1,0,Zookeeper.log,0.0,4.0,19:00:00-20:00:00,374
7628,19:01:46,0,1,0,0,0,Proxifier.log,0.0,2.0,19:00:00-20:00:00,374
7629,19:01:46,0,1,0,0,0,Proxifier.log,0.0,2.0,19:00:00-20:00:00,374
11006,19:29:40,0,1,0,1,0,Zookeeper.log,0.0,4.0,19:00:00-20:00:00,374


# Find the time range having minimum log count

In [712]:
rslt_df = df.loc[(df['time_range_count'] == minvalue )]
rslt_df.head(25)

,time,info,error,debug,warn,exception,filename,timestamp,time_count,time_range,time_range_count
23975,08:52:16,1,1,0,0,0,BGL.log,0.0,1.0,08:00:00-09:00:00,13
23993,08:47:22,1,1,0,0,0,BGL.log,0.0,2.0,08:00:00-09:00:00,13
26580,08:54:17,0,1,0,0,0,Apache.log,0.0,1.0,08:00:00-09:00:00,13
13398,08:35:08,0,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
13392,08:32:40,1,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
13396,08:33:25,0,1,0,0,0,Mac.log,0.0,1.0,08:00:00-09:00:00,13
9652,08:58:45,0,1,0,0,0,HPC.log,0.0,1.0,08:00:00-09:00:00,13
23992,08:20:11,1,1,0,0,0,BGL.log,0.0,1.0,08:00:00-09:00:00,13
9651,08:35:05,0,1,0,0,0,HPC.log,0.0,1.0,08:00:00-09:00:00,13
9907,08:43:00,0,1,0,0,0,HPC.log,0.0,1.0,08:00:00-09:00:00,13


## Get final output 

In [713]:
rslt_df['time_range'].values[0]

'08:00:00-09:00:00'

#  Train and Test

## Remove non required columns

In [714]:
df.drop(['info'], axis=True, inplace=True)
df.drop("filename", axis=1, inplace=True)
df.drop(['time'], axis=True, inplace=True)

In [715]:
df.head(5)

,error,debug,warn,exception,timestamp,time_count,time_range,time_range_count
23975,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
23993,1,0,0,0,0.0,2.0,08:00:00-09:00:00,13
26580,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
13398,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
13392,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13


In [716]:
#df.time_range_count.value_counts()

In [717]:
df

,error,debug,warn,exception,timestamp,time_count,time_range,time_range_count
23975,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
23993,1,0,0,0,0.0,2.0,08:00:00-09:00:00,13
26580,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
13398,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
13392,1,0,0,0,0.0,1.0,08:00:00-09:00:00,13
...,...,...,...,...,...,...,...,...
11088,1,0,1,0,0.0,4.0,19:00:00-20:00:00,374
7628,1,0,0,0,0.0,2.0,19:00:00-20:00:00,374
7629,1,0,0,0,0.0,2.0,19:00:00-20:00:00,374
11006,1,0,1,0,0.0,4.0,19:00:00-20:00:00,374


In [718]:
df.drop("time_range", axis=1, inplace=True)

X = df.drop("time_count", axis=1)
y = df["time_count"]

In [719]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

In [720]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [721]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

In [722]:
logmodel.fit(X_train, y_train)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [723]:
predictions = logmodel.predict(X_test)

In [724]:
from sklearn.metrics import classification_report
classification_report(y_test, predictions)


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'              precision    recall  f1-score   support\n\n         0.0       0.00      0.00      0.00        29\n         1.0       0.00      0.00      0.00       302\n         2.0       0.00      0.00      0.00       101\n         3.0       0.00      0.00      0.00        55\n         4.0       0.00      0.00      0.00        85\n         5.0       0.05      0.80      0.09        45\n         6.0       0.00      0.00      0.00        33\n         7.0       0.00      0.00      0.00        14\n         8.0       0.00      0.00      0.00        43\n         9.0       0.00      0.00      0.00         6\n        10.0       0.00      0.00      0.00        10\n        11.0       0.00      0.00      0.00         3\n        12.0       0.00      0.00      0.00         6\n        13.0       0.00      0.00      0.00         3\n        14.0       0.00      0.00      0.00         9\n        15.0       0.00      0.00      0.00         6\n        17.0       0.00      0.00      0.00         3\n       

In [725]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Accuracy Check


In [726]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)


0.06941176470588235